# PCNSL Data Access Tutorial

This notebook demonstrates how to access and work with CNS lymphoma MRI data from the PCNSL dataset located at `/working/rauschecker1/pcnsl/Box`.

## Table of Contents

0. [Environment Setup](#0-environment-setup)
1. [Loading Anatomy Images](#1-loading-anatomy-images)
2. [Loading Statistics](#2-loading-statistics)
3. [Loading Skullstripped Images and Lesion Overlays](#3-loading-skullstripped-images-and-lesion-overlays)

---
## 0. Environment Setup

### Installing and Activating the Virtual Environment

This project uses `uv` for dependency management with Python 3.11. Follow these steps to set up your environment:

#### Option A: Using `uv` (Recommended)

```bash
# Navigate to the brain_plotting directory
cd /home/mromano/research/cnslymphoma-genetics/brain_plotting

# Install uv if you don't have it
curl -LsSf https://astral.sh/uv/install.sh | sh

# Create and sync the virtual environment
uv sync

# Activate the virtual environment
source .venv/bin/activate
```

#### Option B: Using pip with venv

```bash
# Navigate to the brain_plotting directory
cd /home/mromano/research/cnslymphoma-genetics/brain_plotting

# Create a virtual environment (if .venv doesn't exist)
python3.11 -m venv .venv

# Activate the virtual environment
source .venv/bin/activate

# Install dependencies from pyproject.toml
pip install nibabel nilearn pandas numpy matplotlib jupyter
```

#### For Jupyter Notebook Users

Make sure to register the kernel:

```bash
# With venv activated
pip install ipykernel
python -m ipykernel install --user --name=brain-plotting --display-name="Brain Plotting (Python 3.11)"
```

Then select "Brain Plotting (Python 3.11)" as your kernel in Jupyter.

### Import Required Libraries

In [ ]:
# Standard library
import sys
from pathlib import Path

# Add the tutorials directory to the path for imports
tutorials_dir = Path(".").resolve()
if str(tutorials_dir) not in sys.path:
    sys.path.insert(0, str(tutorials_dir))

# Third-party
import numpy as np
import pandas as pd
import nibabel as nib
import matplotlib.pyplot as plt
from nilearn import plotting

# Local module
from pcnsl_data_loader import (
    PCNSLDataLoader,
    load_all_summary_statistics,
    load_all_individual_lesions,
    load_all_radiomics,
)

# Display settings
%matplotlib inline
plt.rcParams['figure.figsize'] = [12, 8]
pd.set_option('display.max_columns', 20)

print("All imports successful!")

In [ ]:
# Initialize the data loader
# The default path is /working/rauschecker1/pcnsl/Box
loader = PCNSLDataLoader()

# List available subjects
subjects = loader.list_subjects()
print(f"Found {len(subjects)} subjects")
print(f"First 10 subjects: {subjects[:10]}")

---
## 1. Loading Anatomy Images

Anatomy images are stored in the BIDS format under:
```
/working/rauschecker1/pcnsl/Box/sub-XXXX/ses-YYYY/anat/
```

Each subject has three MRI sequences:
- **T1w**: T1-weighted structural image
- **ce-gadolinium_T1w**: Gadolinium-enhanced (post-contrast) T1-weighted image  
- **FLAIR**: Fluid-attenuated inversion recovery image

### 1.1 List Available Anatomy Images for a Subject

In [ ]:
# List anatomy images for a specific subject
subject = "sub-0005"
session = "ses-0001"

anatomy_files = loader.list_anatomy_images(subject, session)
print(f"Anatomy images for {subject}/{session}:")
for f in anatomy_files:
    print(f"  - {f.name}")

### 1.2 Load a Single Anatomy Image

In [ ]:
# Load the FLAIR image
flair_img = loader.load_anatomy_image(subject, session, sequence="FLAIR")

print(f"FLAIR image shape: {flair_img.shape}")
print(f"Voxel dimensions: {flair_img.header.get_zooms()}")
print(f"Data type: {flair_img.get_data_dtype()}")

# Display the image
plotting.plot_anat(flair_img, title=f"{subject} FLAIR", display_mode='ortho')
plt.show()

In [ ]:
# Load the post-contrast T1 image
t1_post = loader.load_anatomy_image(subject, session, sequence="ce-gadolinium_T1w")

print(f"T1-Post image shape: {t1_post.shape}")

# Display the image
plotting.plot_anat(t1_post, title=f"{subject} T1-Post (Gadolinium)", display_mode='ortho')
plt.show()

### 1.3 Load All Anatomy Images at Once

In [ ]:
# Load all anatomy images as a dictionary
all_anatomy = loader.load_anatomy_images(subject, session)

print("Loaded sequences:")
for seq_name, img in all_anatomy.items():
    print(f"  {seq_name}: shape={img.shape}")

In [ ]:
# Visualize all three sequences side by side
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

sequences = ['T1w', 'ce-gadolinium_T1w', 'FLAIR']
titles = ['T1-Weighted', 'T1-Post (Gadolinium)', 'FLAIR']

for ax, seq, title in zip(axes, sequences, titles):
    img = all_anatomy[seq]
    # Get middle slice
    data = img.get_fdata()
    mid_slice = data.shape[2] // 2
    ax.imshow(np.rot90(data[:, :, mid_slice]), cmap='gray')
    ax.set_title(title)
    ax.axis('off')

plt.suptitle(f"{subject} Anatomy Images (Axial View)", fontsize=14)
plt.tight_layout()
plt.show()

---
## 2. Loading Statistics

Statistics are stored in CSV files under:
```
/working/rauschecker1/pcnsl/Box/derivatives/pyalfe/sub-XXXX/ses-YYYY/{auto|human}/statistics/
```

Three types of statistics are available:
- **SummaryLesions**: Aggregate lesion statistics per subject
- **IndividualLesions**: Per-lesion statistics (one row per lesion)
- **radiomics**: PyRadiomics texture features

Each type is available for both **FLAIR** and **T1Post** modalities.

### 2.1 Load Statistics for a Single Subject

In [ ]:
# Load FLAIR SummaryLesions for a single subject
summary_single = loader.load_statistics_single(
    subject="sub-0001",
    session="ses-0001",
    stats_type="SummaryLesions",
    modality="FLAIR",
    processing="auto"
)

print(f"Summary statistics shape: {summary_single.shape}")
print(f"\nColumns: {list(summary_single.columns)[:10]}...")  # First 10 columns
summary_single.head()

In [ ]:
# Load Individual Lesion statistics
individual_single = loader.load_statistics_single(
    subject="sub-0001",
    stats_type="IndividualLesions",
    modality="FLAIR"
)

print(f"Individual lesions shape: {individual_single.shape}")
print(f"Number of lesions found: {len(individual_single)}")
individual_single.head()

### 2.2 Load Statistics for Multiple Subjects

In [ ]:
# Load summary statistics for a list of subjects
selected_subjects = ["sub-0001", "sub-0002", "sub-0003", "sub-0005", "sub-0010"]

multi_summary = loader.load_statistics(
    subjects=selected_subjects,
    stats_type="SummaryLesions",
    modality="FLAIR",
    processing="auto"
)

print(f"Loaded statistics for {len(multi_summary)} subjects")
multi_summary[['subject', 'total_lesion_volume', 'lesion_volume_in_white_matter']].head(10)

### 2.3 Load Statistics for All Subjects

In [ ]:
# Load all summary statistics using the convenience function
all_summary = load_all_summary_statistics(modality="FLAIR", processing="auto")

print(f"Loaded summary statistics for {len(all_summary)} subjects")
print(f"Columns: {len(all_summary.columns)}")
all_summary.head()

In [ ]:
# Basic statistics on lesion volumes
print("Lesion Volume Statistics (FLAIR):")
print("=" * 40)
print(f"Mean total lesion volume: {all_summary['total_lesion_volume'].mean():.2f} mm³")
print(f"Median total lesion volume: {all_summary['total_lesion_volume'].median():.2f} mm³")
print(f"Min: {all_summary['total_lesion_volume'].min():.2f} mm³")
print(f"Max: {all_summary['total_lesion_volume'].max():.2f} mm³")

In [ ]:
# Visualize lesion volume distribution
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Histogram
axes[0].hist(all_summary['total_lesion_volume'], bins=30, edgecolor='black', alpha=0.7)
axes[0].set_xlabel('Total Lesion Volume (mm³)')
axes[0].set_ylabel('Count')
axes[0].set_title('Distribution of Total Lesion Volume')

# Box plot by tissue type
tissue_cols = [
    'lesion_volume_in_white_matter',
    'lesion_volume_in_Cortical Gray Matter',
    'lesion_volume_in_Deep Gray Matter',
    'lesion_volume_in_CorpusCallosum'
]
tissue_data = all_summary[tissue_cols].copy()
tissue_data.columns = ['White Matter', 'Cortical GM', 'Deep GM', 'Corpus Callosum']
tissue_data.boxplot(ax=axes[1], rot=45)
axes[1].set_ylabel('Lesion Volume (mm³)')
axes[1].set_title('Lesion Volume by Tissue Type')

plt.tight_layout()
plt.show()

### 2.4 Compare FLAIR vs T1Post Statistics

In [ ]:
# Load T1Post statistics for comparison
all_summary_t1post = load_all_summary_statistics(modality="T1Post", processing="auto")

# Compare lesion volumes
comparison = pd.DataFrame({
    'subject': all_summary['subject'],
    'FLAIR_volume': all_summary['total_lesion_volume'],
    'T1Post_volume': all_summary_t1post['total_lesion_volume']
})

# Scatter plot
plt.figure(figsize=(8, 8))
plt.scatter(comparison['FLAIR_volume'], comparison['T1Post_volume'], alpha=0.6)
max_val = max(comparison['FLAIR_volume'].max(), comparison['T1Post_volume'].max())
plt.plot([0, max_val], [0, max_val], 'r--', label='y=x')
plt.xlabel('FLAIR Lesion Volume (mm³)')
plt.ylabel('T1Post Lesion Volume (mm³)')
plt.title('FLAIR vs T1Post Lesion Volume')
plt.legend()
plt.axis('equal')
plt.tight_layout()
plt.show()

### 2.5 Load Individual Lesion Data for All Subjects

In [ ]:
# Load individual lesion statistics for all subjects
all_individual = load_all_individual_lesions(modality="FLAIR", processing="auto")

print(f"Total individual lesions: {len(all_individual)}")
print(f"Unique subjects: {all_individual['subject'].nunique()}")
print(f"\nLesions per subject:")
print(all_individual.groupby('subject').size().describe())

### 2.6 Load Radiomics Features

In [ ]:
# Load radiomics for a single subject
radiomics_single = loader.load_statistics_single(
    subject="sub-0001",
    stats_type="radiomics",
    modality="FLAIR"
)

print(f"Radiomics features shape: {radiomics_single.shape}")
print(f"\nSample features:")
radiomics_single.head(10)

---
## 3. Loading Skullstripped Images and Lesion Overlays

Processed images are stored under:
```
/working/rauschecker1/pcnsl/Box/derivatives/pyalfe/sub-XXXX/ses-YYYY/{auto|human}/
├── skullstripped/
│   ├── lesions_FLAIR_space/    # Images registered to FLAIR
│   └── lesions_T1Post_space/   # Images registered to T1Post
└── masks/
    └── lesions_seg_comp/       # Lesion segmentation masks
```

### Available Sequences per Space:
- T1, T1Post, FLAIR, ADC

### 3.1 Load Skullstripped Images in FLAIR Space

In [ ]:
# List available skullstripped images
subject = "sub-0001"

ss_files = loader.list_skullstripped_images(subject, processing="auto", space="FLAIR")
print(f"Skullstripped images in FLAIR space for {subject}:")
for f in ss_files:
    print(f"  - {f.name}")

In [ ]:
# Load a specific skullstripped image
flair_ss = loader.load_skullstripped_image(
    subject,
    processing="auto",
    space="FLAIR",
    sequence="FLAIR"
)

print(f"Skullstripped FLAIR shape: {flair_ss.shape}")

# Display
plotting.plot_anat(flair_ss, title=f"{subject} Skullstripped FLAIR", display_mode='ortho')
plt.show()

In [ ]:
# Load all skullstripped images in FLAIR space
all_ss_flair = loader.load_skullstripped_images(subject, processing="auto", space="FLAIR")

print("Loaded skullstripped sequences:")
for seq, img in all_ss_flair.items():
    print(f"  {seq}: shape={img.shape}")

### 3.2 Load Skullstripped Images in T1Post Space

In [ ]:
# Load T1Post in T1Post space
t1post_ss = loader.load_skullstripped_image(
    subject,
    processing="auto",
    space="T1Post",
    sequence="T1Post"
)

print(f"Skullstripped T1Post shape: {t1post_ss.shape}")

# Display
plotting.plot_anat(t1post_ss, title=f"{subject} Skullstripped T1Post", display_mode='ortho')
plt.show()

### 3.3 Load and Overlay Lesion Masks

In [ ]:
# Load the FLAIR lesion mask
flair_mask = loader.load_lesion_mask(subject, processing="auto", modality="FLAIR")

mask_data = flair_mask.get_fdata()
print(f"Mask shape: {flair_mask.shape}")
print(f"Number of lesion voxels: {np.sum(mask_data > 0)}")
print(f"Unique mask values: {np.unique(mask_data)}")

In [ ]:
# Overlay FLAIR lesions on FLAIR image using nilearn
plotting.plot_roi(
    flair_mask,
    bg_img=flair_ss,
    title=f"{subject} FLAIR with Lesion Overlay",
    display_mode='ortho',
    alpha=0.5,
    cmap='hot'
)
plt.show()

In [ ]:
# Use the convenience method to load both image and mask together
img, mask = loader.load_image_with_mask(subject, processing="auto", modality="FLAIR")

# Plot with different display modes
fig = plt.figure(figsize=(15, 10))

# Mosaic view
plotting.plot_roi(
    mask,
    bg_img=img,
    title=f"{subject} FLAIR Lesions (Mosaic)",
    display_mode='mosaic',
    cut_coords=8,
    alpha=0.5,
    cmap='autumn'
)
plt.show()

### 3.4 Overlay T1Post Lesions on T1Post Image

In [ ]:
# Load T1Post image and mask
t1post_img, t1post_mask = loader.load_image_with_mask(subject, processing="auto", modality="T1Post")

print(f"T1Post image shape: {t1post_img.shape}")
print(f"T1Post mask shape: {t1post_mask.shape}")

# Overlay
plotting.plot_roi(
    t1post_mask,
    bg_img=t1post_img,
    title=f"{subject} T1Post with Lesion Overlay",
    display_mode='ortho',
    alpha=0.5,
    cmap='hot'
)
plt.show()

### 3.5 Compare FLAIR and T1Post Lesions Side by Side

In [ ]:
# Custom visualization comparing both modalities
fig, axes = plt.subplots(2, 3, figsize=(15, 10))

# Get data
flair_data = flair_ss.get_fdata()
flair_mask_data = flair_mask.get_fdata()
t1post_data = t1post_ss.get_fdata()
t1post_mask_data = t1post_mask.get_fdata()

# Find slices with lesions for FLAIR
lesion_slices = np.where(flair_mask_data.sum(axis=(0, 1)) > 0)[0]
if len(lesion_slices) > 0:
    mid_lesion_slice = lesion_slices[len(lesion_slices) // 2]
else:
    mid_lesion_slice = flair_data.shape[2] // 2

# FLAIR row
axes[0, 0].imshow(np.rot90(flair_data[:, :, mid_lesion_slice]), cmap='gray')
axes[0, 0].set_title('FLAIR Image')
axes[0, 0].axis('off')

axes[0, 1].imshow(np.rot90(flair_mask_data[:, :, mid_lesion_slice]), cmap='hot')
axes[0, 1].set_title('FLAIR Lesion Mask')
axes[0, 1].axis('off')

# Overlay
axes[0, 2].imshow(np.rot90(flair_data[:, :, mid_lesion_slice]), cmap='gray')
masked = np.ma.masked_where(flair_mask_data[:, :, mid_lesion_slice] == 0, 
                            flair_mask_data[:, :, mid_lesion_slice])
axes[0, 2].imshow(np.rot90(masked), cmap='hot', alpha=0.6)
axes[0, 2].set_title('FLAIR with Overlay')
axes[0, 2].axis('off')

# T1Post row - need to find corresponding slice (may be different shape)
t1_slice = min(mid_lesion_slice, t1post_data.shape[2] - 1)
lesion_slices_t1 = np.where(t1post_mask_data.sum(axis=(0, 1)) > 0)[0]
if len(lesion_slices_t1) > 0:
    t1_slice = lesion_slices_t1[len(lesion_slices_t1) // 2]

axes[1, 0].imshow(np.rot90(t1post_data[:, :, t1_slice]), cmap='gray')
axes[1, 0].set_title('T1Post Image')
axes[1, 0].axis('off')

axes[1, 1].imshow(np.rot90(t1post_mask_data[:, :, t1_slice]), cmap='hot')
axes[1, 1].set_title('T1Post Lesion Mask')
axes[1, 1].axis('off')

# Overlay
axes[1, 2].imshow(np.rot90(t1post_data[:, :, t1_slice]), cmap='gray')
masked_t1 = np.ma.masked_where(t1post_mask_data[:, :, t1_slice] == 0, 
                                t1post_mask_data[:, :, t1_slice])
axes[1, 2].imshow(np.rot90(masked_t1), cmap='hot', alpha=0.6)
axes[1, 2].set_title('T1Post with Overlay')
axes[1, 2].axis('off')

plt.suptitle(f'{subject} Lesion Comparison', fontsize=14)
plt.tight_layout()
plt.show()

### 3.6 Process Multiple Subjects

In [ ]:
# List subjects with human (manual) annotations
subjects_with_human = loader.list_subjects_with_processing(processing="human")
print(f"Subjects with manual annotations: {len(subjects_with_human)}")
print(subjects_with_human[:10])

In [ ]:
# Batch visualization of multiple subjects
subjects_to_show = subjects_with_human[:4] if len(subjects_with_human) >= 4 else subjects[:4]

fig, axes = plt.subplots(2, 4, figsize=(16, 8))

for idx, subj in enumerate(subjects_to_show):
    try:
        # Try human processing first, fall back to auto
        try:
            img, mask = loader.load_image_with_mask(subj, processing="human", modality="FLAIR")
            processing_type = "human"
        except FileNotFoundError:
            img, mask = loader.load_image_with_mask(subj, processing="auto", modality="FLAIR")
            processing_type = "auto"
        
        img_data = img.get_fdata()
        mask_data = mask.get_fdata()
        
        # Find slice with most lesions
        lesion_counts = mask_data.sum(axis=(0, 1))
        best_slice = np.argmax(lesion_counts)
        
        # Image
        axes[0, idx].imshow(np.rot90(img_data[:, :, best_slice]), cmap='gray')
        axes[0, idx].set_title(f'{subj}\n({processing_type})')
        axes[0, idx].axis('off')
        
        # Overlay
        axes[1, idx].imshow(np.rot90(img_data[:, :, best_slice]), cmap='gray')
        masked = np.ma.masked_where(mask_data[:, :, best_slice] == 0, 
                                    mask_data[:, :, best_slice])
        axes[1, idx].imshow(np.rot90(masked), cmap='autumn', alpha=0.6)
        axes[1, idx].axis('off')
        
    except Exception as e:
        axes[0, idx].text(0.5, 0.5, f'Error: {str(e)[:30]}', 
                          ha='center', va='center', transform=axes[0, idx].transAxes)
        axes[0, idx].set_title(subj)
        axes[1, idx].axis('off')

plt.suptitle('FLAIR Images with Lesion Overlays', fontsize=14)
plt.tight_layout()
plt.show()

---
## Summary

This tutorial covered:

1. **Environment Setup**: Installing and activating the virtual environment using `uv` or `pip`

2. **Loading Anatomy Images**: 
   - Listing available subjects and sessions
   - Loading individual sequences (T1w, T1-Post, FLAIR)
   - Loading all anatomy images at once

3. **Loading Statistics**:
   - Single subject statistics (SummaryLesions, IndividualLesions, radiomics)
   - Multiple subject statistics
   - All subjects (convenience functions)
   - Comparing FLAIR vs T1Post statistics

4. **Skullstripped Images and Overlays**:
   - Loading skullstripped images in FLAIR or T1Post space
   - Loading lesion segmentation masks
   - Creating lesion overlays for visualization
   - Comparing FLAIR and T1Post lesions

### Key Classes and Functions

```python
# If running from the tutorials directory:
from pcnsl_data_loader import (
    PCNSLDataLoader,              # Main class for data access
    load_all_summary_statistics,  # Load all subjects' summary stats
    load_all_individual_lesions,  # Load all individual lesion data
    load_all_radiomics,           # Load all radiomics features
)

# Or if running from brain_plotting root:
from tutorials import PCNSLDataLoader
```

### Directory Structure Reference

```
/working/rauschecker1/pcnsl/Box/
├── sub-XXXX/ses-YYYY/anat/                    # Raw anatomy images
└── derivatives/pyalfe/sub-XXXX/ses-YYYY/
    ├── auto/                                   # Automated processing
    │   ├── statistics/lesions_{type}/         # Statistics CSVs
    │   ├── skullstripped/lesions_{space}_space/  # Skullstripped images
    │   └── masks/lesions_seg_comp/            # Lesion masks
    └── human/                                  # Manual annotations (same structure)
```